In [ ]:
import os.path

import xarray as xr
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.colors as mpl_c
import matplotlib.patches as mpl_p
import cartopy.crs as ccrs

In [ ]:
plt.style.use('paper')
plt.style.use('egu_journals')

In [ ]:
rotated_pole = ccrs.RotatedPole(pole_longitude=-171.0, pole_latitude=41.5)
plate_carree = ccrs.PlateCarree()

In [ ]:
base_path = '/work/um0203/u300636/for2131/runs/'

In [ ]:
topo_path = os.path.join(base_path, 'inputs', 'clm', 'topodata_0302x0267.nc')
ds_topo = xr.open_dataset(topo_path)

In [ ]:
ds_topo['lsmlat'] = ds_topo['LATIXY'].mean('lsmlon')
ds_topo['lsmlon'] = ds_topo['LONGXY'].mean('lsmlat')

In [ ]:
stations_path = os.path.join(base_path, 'utilities', 'stations.hd5')
ds_stations = pd.read_hdf(stations_path)

In [ ]:
rect_left_xy = [ds_topo['lsmlon'].min(), ds_topo['lsmlat'].min()]
rect_left_wh = [ds_topo['lsmlon'][30]-ds_topo['lsmlon'].min(), ds_topo['lsmlat'].max()-ds_topo['lsmlat'].min()]

rect_right_xy = [ds_topo['lsmlon'][-30], ds_topo['lsmlat'].min()]
rect_right_wh = [ds_topo['lsmlon'].max()-ds_topo['lsmlon'][-30], ds_topo['lsmlat'].max()-ds_topo['lsmlat'].min()]

rect_top_xy = [ds_topo['lsmlon'][30], ds_topo['lsmlat'][-30]]
rect_top_wh = [ds_topo['lsmlon'][-30]-ds_topo['lsmlon'][30], ds_topo['lsmlat'].max()-ds_topo['lsmlat'][-30]]

rect_bottom_xy = [ds_topo['lsmlon'][30], ds_topo['lsmlat'].min()]
rect_bottom_wh = [ds_topo['lsmlon'][-30]-ds_topo['lsmlon'][30], ds_topo['lsmlat'][30]-ds_topo['lsmlat'].min()]

In [ ]:
idx = 17

figsize = [s*0.48 for s in plt.rcParams['figure.figsize']]
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot()
norm = mpl_c.Normalize(vmin=0.,vmax=1250.)
cf = ax.contourf(ds_topo['lsmlon'].values, ds_topo['lsmlat'].values, ds_topo['TOPO'].values, levels=np.arange(0, 1800, 50), cmap='pink_r', norm=norm)
ax.scatter(ds_stations['Länge'], ds_stations['Breite'], s=5, marker='.', color='black', zorder=99999)
#ax.scatter(ds_stations['Länge'].iloc[idx], ds_stations['Breite'].iloc[idx], s=5, marker='x', color='red', zorder=99999)

ax.add_patch(
    mpl_p.Rectangle(xy=rect_left_xy, width=rect_left_wh[0], height=rect_left_wh[1],
                    facecolor='black', alpha=0.2)
)
ax.add_patch(
    mpl_p.Rectangle(xy=rect_right_xy, width=rect_right_wh[0], height=rect_right_wh[1],
                    facecolor='black', alpha=0.2)
)
ax.add_patch(
    mpl_p.Rectangle(xy=rect_top_xy, width=rect_top_wh[0], height=rect_top_wh[1],
                    facecolor='black', alpha=0.2)
)
ax.add_patch(
    mpl_p.Rectangle(xy=rect_bottom_xy, width=rect_bottom_wh[0], height=rect_bottom_wh[1],
                    facecolor='black', alpha=0.2)
)


cbar = fig.colorbar(cf, fraction=0.046, pad=0.04)
cbar.set_label('Orography (m)')

ax.set_xlim(7.491, 10.72)
ax.set_xticks(np.arange(7.5, 11, 0.5))
ax.set_xlabel(r'Longitude ($^\circ$)')
ax.set_ylabel(r'Latitude ($^\circ$)')
ax.spines['right'].set_visible(True)
ax.spines['top'].set_visible(True)

fig.savefig('../figures/fig_01_orography.png', dpi=300)
plt.show()